In [17]:
%matplotlib inline

In [33]:
import networkx as nx
import pandas as pd
import metis
import pickle as pkl
import numpy as np

from graph_helpers import load_graph_by_name
from collections import defaultdict
from helpers import infected_nodes

In [19]:
def get_cut(nodes):
    node_set = set(nodes)
    n_cut = 0
    for u in nodes:
        for v in g.neighbors(u):        
            if v not in node_set:
                n_cut += 1
    return n_cut       

In [20]:
def cut_from_each_node(nodes):
    node_set = set(nodes)
    cut = {}
    for u in nodes:
        cut[u] = 0
        for v in g.neighbors(u):        
            if v not in node_set:
                cut[u] += 1
    return cut       

In [21]:
gt = load_graph_by_name('grqc')

load graph from data/grqc/graph.gt


In [22]:
list(gt.vertex(0).out_neighbours())

[<Vertex object with index '3605' at 0x7f42851fa9f0>]

In [23]:
g = nx.read_graphml('data/grqc/graph.graphml')


In [24]:
new_g = nx.Graph()
for u, v in g.edges_iter():
    new_g.add_edge(int(u), int(v))

In [25]:
g = new_g

In [26]:
assert gt.num_vertices() == g.number_of_nodes()
assert gt.num_edges() == g.number_of_edges()

for u, v in g.edges_iter():
    assert gt.edge(u, v) is not None


In [27]:
k = 50
cuts, comm = metis.part_graph(g, k)

In [28]:
comm_list = defaultdict(list)
for v, c in enumerate(comm):
    comm_list[c].append(v)

In [29]:
comm_cuts = {c: get_cut(nodes) for c, nodes in comm_list.items()}
records = [{'id': c, 'nodes': comm_list[c], 'cut': comm_cuts[c]} for c in comm_list]
df = pd.DataFrame.from_records(records)
df['size'] = df['nodes'].apply(len)
df['ratio'] = df['size'] / df['cut']

In [30]:
top_comms = df.sort_values('ratio', ascending=False).head(12)
top_comms.index

Int64Index([6, 27, 35, 25, 3, 5, 7, 15, 44, 40, 17, 31], dtype='int64')

In [31]:
top_comms

,cut,id,nodes,size,ratio
6,23,6,"[16, 71, 132, 147, 151, 165, 186, 231, 328, 42...",84,3.652174
27,27,27,"[43, 54, 86, 137, 143, 152, 171, 174, 178, 225...",82,3.037037
35,30,35,"[50, 100, 190, 197, 220, 229, 251, 300, 311, 4...",83,2.766667
25,31,25,"[39, 56, 57, 83, 98, 141, 235, 264, 266, 305, ...",83,2.677419
3,32,3,"[18, 30, 31, 82, 145, 255, 473, 563, 582, 600,...",83,2.593750
5,35,5,"[15, 61, 139, 172, 196, 198, 214, 227, 275, 28...",83,2.371429
7,36,7,"[53, 59, 93, 99, 114, 134, 158, 179, 188, 319,...",82,2.277778
15,42,15,"[0, 10, 17, 25, 47, 78, 95, 117, 217, 283, 295...",85,2.023810
44,46,44,"[9, 112, 113, 126, 130, 206, 208, 434, 533, 62...",84,1.826087
40,48,40,"[12, 35, 123, 144, 194, 207, 240, 247, 303, 42...",85,1.770833


In [35]:
for i, cid in enumerate(top_comms.index): 
    c = np.ones(g.number_of_nodes()) * -1
    nodes = df.iloc[cid]['nodes']
    cd = cut_from_each_node(nodes)
    source = min(nodes, key=cd.__getitem__)
    c[nodes] = 1    
    c[source] = 0
    
    for j in range(8):
        obs = np.random.permutation(nodes)[:int(len(nodes) * 0.2)]
        pkl.dump((obs, c), open('cascade-weighted/grqc-sto-mcom-s0.02-o0.2/{}.pkl'.format(i * 8 + j), 'wb'))

In [257]:

cuts = []
for i in np.arange(0, 96, 8):
    _, c = pkl.load(open('cascade-weighted/grqc-msi-s0.02-o0.2/{}.pkl'.format(i), 'rb'))
    nodes = list(infected_nodes(c))
    cuts.append(get_cut(nodes))

In [258]:
print('mean cut', np.mean(cuts))

mean cut 481.3333333333333


In [260]:
print('mean cut in this case', top_comms['cut'].mean())

mean cut in this case 41.333333333333336


In [270]:
from glob import glob
def mean_runnig_time(d):
    return np.mean([pkl.load(open(p, 'rb'))['time_elapsed'] for p in glob(d + '/*.meta.pkl')])

In [271]:
mean_runnig_time('outputs/queries-weighted/grqc-msi-s0.02-o0.2/loop_erased/entropy')

269.7858134607474

In [273]:
mean_runnig_time('outputs/queries-weighted/grqc-mcomm-s0.02-o0.2/loop_erased/entropy')

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan